In [1]:
import pandas as pd
import numpy as np
import nltk
from textblob import TextBlob

In [ ]:
dados = pd.read_csv('../data/noticias_estadao.csv')

### Visão geral dos dados

In [38]:
dados.head()

(7643, 3)

In [39]:
print("Os dados tem %d observações e %d colunas" % dados.shape)

Os dados tem 7643 observações e 3 colunas


### Índices invertidos

A função calcula os índices invertidos para a base de dados, considerando as colunas **titulo** e **conteudo** para a construção. Não há diferenciação entre letras maiúsculas e minúsculas nas palavras (são transformadas em minúsculo no algoritmo).

A estrutura da dados utilizada é um dicionário onde as chaves são as palavras e o conteúdo são sets contendo os índices de cada notícia onde essa palavra aparece.

In [3]:
def calcula_indices_invertidos(data):
    indices = {}
    for index, row in data.iterrows():
        palavras_titulo = TextBlob(row["titulo"]).words
        palavras_conteudo = TextBlob(row["conteudo"]).words
        
        for palavra in palavras_titulo:
            palavra_low = palavra.lower()
            if(palavra_low in indices):
                indices[palavra_low].add(row["idNoticia"])
            else:
                indices[palavra_low] = {row["idNoticia"]}
                
        for palavra in palavras_conteudo:
            palavra_low = palavra.lower()
            if(palavra_low in indices):
                indices[palavra_low].add(row["idNoticia"])
            else:
                indices[palavra_low] = {row["idNoticia"]}
    return(indices)

indices_invertidos = calcula_indices_invertidos(dados)

### Algoritmos de Busca

#### Busca um termo
A busca de apenas um termo é feita através de uma consulta simples no dicionário de índices invertidos.
O método retorna um set vazio se a palavra não for encontrada.

In [30]:
def busca_um_termo(termo):
    try:
        return(indices_invertidos[termo])
    except KeyError:
        return(set())

#### Busca AND
A busca AND de 2 termos é feita usando a operação de intersecção de conjuntos entre duas consultas diretas no dicionário de índices invertidos.

O método retorna um set vazio se uma das palavras não for encontrada.

In [6]:
def busca_and(termo1, termo2):
    try:
        busca1 = indices_invertidos[termo1]
        busca2 = indices_invertidos[termo2]
        return(busca1 & busca2)
    except KeyError:
        return(set())

#### Busca OR
A busca OR de 2 termos é feita usando a operação de união de conjuntos entre duas consultas diretas no dicionário de índices invertidos.

In [7]:
def busca_or(termo1, termo2):
    try:
        busca1 = indices_invertidos[termo1]
    except KeyError:
        busca1 = set()
        
    try:
        busca2 = indices_invertidos[termo2]
    except KeyError:
        busca2 = set()
        
    return(busca1 | busca2)

#### Busca booleana
O algoritmo generaliza as consultas de busca de um termo, and e or, além disso, trata casos em que a entrada não corresponde à uma entrada válida, como número inválido de argumentos ou uso de um operador inváĺido.

In [8]:
def busca_booleana(*args):
    if (len(args) != 3 and len(args) != 1):
        return("Número inválido de argumentos")
    if(len(args) == 1):
        termo = args[0].lower()
        return(busca_um_termo(termo))
    elif(args[2] == "AND"):
        termo1 = args[0].lower()
        termo2 = args[1].lower()
        return(busca_and(termo1, termo2))
    elif(args[2] == "OR"):
        termo1 = args[0].lower()
        termo2 = args[1].lower()
        return(busca_or(termo1, termo2))
    else:
        return("Operador inválido, use 'AND' ou 'OR'")

### Questão Bônus - Busca AND de n termos

O método abaixo realiza a busca AND de n termosele realiza uma otimização fazer a primeira operação de AND entre o maior e o menor conjunto de id's, deste modo, o maior tamanho possível para a resultado final é o tamanho da menor lista, o que reduz o tempo de cálculo dos AND's seguintes.

O método retorna um set vazio se uma das palavras não for encontrada.

In [35]:
def busca_and_n_termos(*args):
    result = []
    for i in range(len(args)):
        try:
            result.append(busca_um_termo(args[i]))
        except KeyError:
            return(set())
        
    result.sort(key = lambda item: len(item))
    busca = result[0] & result[-1]
    
    for i in range(len(result)-1, 1, -1):
        busca = busca & result[i]
        
    return(busca)
    
busca_and_n_termos("presidenciáveis", "aeae")

set()

### Consultas

In [73]:
busca_booleana("Campina", "Grande", "AND")

{1068, 1370, 1770, 1952, 1987, 2763, 2777, 2779, 4802, 5382, 5870, 6694}

In [ ]:
busca_booleana("candidatos")

In [ ]:
busca_booleana("debate", "presidencial", "AND")

In [ ]:
busca_booleana("debate", "presidencial", "OR")

In [34]:
busca_booleana("presidenciáveis", "corruptos", "AND")

set()

In [33]:
busca_booleana("presidenciáveis", "corruptos", "OR")

{68,
 93,
 126,
 149,
 160,
 176,
 180,
 272,
 304,
 330,
 375,
 422,
 426,
 430,
 456,
 497,
 523,
 538,
 539,
 777,
 789,
 841,
 874,
 893,
 1109,
 1129,
 1158,
 1198,
 1325,
 1326,
 1343,
 1428,
 1461,
 1470,
 1481,
 1487,
 1537,
 1639,
 1693,
 1847,
 1851,
 1859,
 2023,
 2024,
 2026,
 2028,
 2030,
 2051,
 2068,
 2069,
 2080,
 2088,
 2093,
 2123,
 2125,
 2144,
 2149,
 2152,
 2225,
 2248,
 2253,
 2285,
 2442,
 2458,
 2471,
 2507,
 2516,
 2571,
 2574,
 2609,
 2628,
 2660,
 2669,
 2672,
 2676,
 2678,
 2686,
 2732,
 2740,
 2764,
 2813,
 2860,
 3042,
 3046,
 3182,
 3188,
 3217,
 3243,
 3248,
 3260,
 3377,
 3391,
 3397,
 3444,
 3446,
 3447,
 3466,
 3479,
 3489,
 3546,
 3607,
 3616,
 3633,
 3641,
 3664,
 3666,
 3684,
 3730,
 3747,
 3767,
 3786,
 3843,
 3860,
 3962,
 4025,
 4034,
 4042,
 4137,
 4170,
 4177,
 4184,
 4199,
 4219,
 4235,
 4258,
 4260,
 4294,
 4488,
 4492,
 4660,
 4677,
 4743,
 4847,
 4926,
 4965,
 5110,
 5115,
 5121,
 5133,
 5237,
 5271,
 5472,
 5587,
 5659,
 5713,
 5895,
 590

In [ ]:
busca_booleana("Belo", "Horizonte", "AND")

In [ ]:
busca_booleana("Belo", "Horizonte", "OR")

### Asserts

In [14]:
assert len(busca_booleana("debate", "presidencial", "OR")) , ""== 1770
assert len(busca_booleana("debate", "presidencial", "AND")) == 201
assert len(busca_booleana("presidenciáveis", "corruptos","OR")) == 164
assert len(busca_booleana("presidenciáveis", "corruptos", "AND"))== 0
assert len(busca_booleana("Belo", "Horizonte", "OR")) == 331
assert len(busca_booleana("Belo", "Horizonte", "AND")) == 242
